In [5]:
from ultralytics import YOLO
import mlflow
import mlflow.pytorch
import cv2
import time
import torch

# ✅ Load trained model
model = YOLO("runs/detect/yolov8m_kitti_enhanced/weights/best.pt")

# ✅ Start MLflow experiment
mlflow.set_experiment("AV_Object_Detection_Monitoring")

with mlflow.start_run(run_name="RealTime_Inference_Monitoring"):
    
    # Log model as artifact
    mlflow.log_param("model_type", "YOLOv8m")
    mlflow.log_artifact("runs/detect/yolov8m_kitti_enhanced/weights/best.pt", artifact_path="model")

    
    # Log static model metrics manually (from your training report)
    mlflow.log_metric("mAP", 0.87)  # Example value, replace with your best mAP
    mlflow.log_metric("precision", 0.85)
    mlflow.log_metric("recall", 0.83)

    # ✅ Real-time monitoring (logs FPS and average detection confidence)
    cap = cv2.VideoCapture(0)  # Or your test video path
    frame_count = 0
    start_time = time.time()
    total_confidence = 0
    total_detections = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, conf=0.3, iou=0.5, imgsz=640, device=0)
        
        for r in results:
            if r.boxes is not None:
                confidences = r.boxes.conf.cpu().numpy()
                total_confidence += sum(confidences)
                total_detections += len(confidences)
                
        frame_count += 1

        # Show live detections
        annotated_frame = results[0].plot()
        cv2.imshow("AV Detection Monitor", annotated_frame)

        # Break on key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # ✅ End of monitoring — calculate and log performance
    end_time = time.time()
    duration = end_time - start_time
    fps = frame_count / duration
    avg_confidence = total_confidence / total_detections if total_detections > 0 else 0

    mlflow.log_metric("FPS", fps)
    mlflow.log_metric("avg_confidence", avg_confidence)
    mlflow.log_metric("total_detections", total_detections)

    print(f"✅ Logged to MLflow ➔ FPS: {fps:.2f}, Avg Confidence: {avg_confidence:.2f}")

    cap.release()
    cv2.destroyAllWindows()


0: 480x640 (no detections), 56.9ms
Speed: 10.8ms preprocess, 56.9ms inference, 78.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 53.3ms
Speed: 4.6ms preprocess, 53.3ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 (no detections), 53.4ms
Speed: 2.8ms preprocess, 53.4ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 (no detections), 51.3ms
Speed: 5.2ms preprocess, 51.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 (no detections), 50.9ms
Speed: 2.5ms preprocess, 50.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Car, 48.5ms
Speed: 3.3ms preprocess, 48.5ms inference, 157.9ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 Car, 48.5ms
Speed: 3.3ms preprocess, 48.5ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 (no detections), 48.3ms
Speed: 2.7ms preprocess, 48.3ms inference, 1.8ms postpro